In [2]:
from pyqlearning.qlearning.greedy_q_learning import GreedyQLearning

In [6]:
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

In [13]:
class PriceReinforcementLearning(GreedyQLearning):
    def __init__(self):
        super().__init__()
        self.set_alpha_value(1.0)
        self.set_epsilon_greedy_rate(0.8)

    def extract_possible_actions(self, state_key):
        return list(filter(lambda x: 1 <= x <= 5, (state_key - 1, state_key, state_key + 1)))

    def observe_reward_value(self, state_key, action_key):
        return 5 - action_key

    def learn(self, a, limit=1000):
        super().learn(a, limit)
        print(self.q_df.sort_values("q_value", ascending=False))

In [14]:
ql = PriceReinforcementLearning()
for i in range(1, 6):
    for j in range(max(1, i - 1), min(6, i + 2)):
        ql.save_q_df(i, j, 1.0)

In [15]:
ql.learn(1, limit=1000)

   state_key  action_key   q_value
0        1.0         1.0  8.000000
0        2.0         1.0  8.000000
0        1.0         2.0  7.000000
0        3.0         2.0  7.000000
0        2.0         2.0  7.000000
0        4.0         3.0  5.500000
0        2.0         3.0  5.500000
0        3.0         4.0  3.750000
0        4.0         4.0  2.000000
0        5.0         4.0  2.000000
0        3.0         3.0  1.000000
0        4.0         5.0  0.999992
0        5.0         5.0  0.500000
